In [1]:
from libraries import *

In [2]:
client = QdrantClient(
    url="https://cecd63ca-9699-4bc0-a60f-3c7e8c768fd3.europe-west3-0.gcp.cloud.qdrant.io:6333",  # Replace with your cloud cluster URL
    api_key="jwv0ETSb8751Q1yrQ3RVSsChFudsv1M63mEvxXXRnLf_ROoeAfe_Wg"  # Replace with your API key
)

collection_name = "LawEmbedding"

In [16]:
# if client.get_collection(collection_name):
#     client.delete_collection(collection_name)  # This will delete the collection if it exists
#     print(f"Collection '{collection_name}' deleted.")

Collection 'LawEmbeddingEUCLIDEUCLID' deleted.


In [3]:
embeddings = np.loadtxt("embeddings-complete-legal-bert-base-uncased.csv", delimiter=',')
df = pd.read_csv("Title18.csv")

In [4]:
embeddings.shape

(1647, 768)

In [5]:
df.shape

(1647, 4)

In [22]:
dimension = embeddings.shape[1]  # Get the dimensionality of the embeddings

# client.recreate_collection(
#     collection_name=collection_name+'COSINE',
#     vectors_config=VectorParams(size=dimension, distance=Distance.COSINE)  # Use COSINE distance for similarity
# )

client.recreate_collection(
    collection_name=collection_name+'EUCLID',
    vectors_config=VectorParams(size=dimension, distance=Distance.EUCLID)  # Use COSINE distance for similarity
)


# Step 5: Insert embeddings row by row into Qdrant
points = [
    {
        "id": idx,  # The row index will be used as the unique ID
        "vector": embeddings.tolist(),  # Convert numpy array to list for Qdrant
        "payload": {
            "section": df['Section'].iloc[idx],
            "url": df['Url'].iloc[idx],
        }
    }
    for idx, embeddings in enumerate(embeddings)
]

# client.upsert(
#     collection_name=collection_name+'COSINE',
#     points=points
# )

client.upsert(
    collection_name=collection_name+'EUCLID',
    points=points
)

# info = client.get_collection(collection_name+'COSINE')
# print(f"Number of vectors in the collection: {info.points_count}")

info = client.get_collection(collection_name+'EUCLID')
print(f"Number of vectors in the collection: {info.points_count}")

C:\Users\npatel237\AppData\Local\Temp\ipykernel_20788\734515275.py:8: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Number of vectors in the collection: 1647


In [6]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def get_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

test = ''' 
Mandatory Restitution.-Upon a conviction under this section, the court shall order restitution under
section 3663A in an amount equal to the total unpaid support obligation as it exists at the time of sentencing.
'''

test_embedding = get_embedding(test, tokenizer, model).cpu().numpy().astype('float32')
test_embedding_flat = test_embedding.flatten().tolist()  # Flatten to 1D list

search_result_cosine = client.search(
    collection_name=collection_name+'COSINE',
    query_vector=test_embedding_flat,  # Use 'query_vector' parameter directly
    limit=5  # Number of nearest neighbors you want to retrieve
)

search_result_euclid = client.search(
    collection_name=collection_name+'EUCLID',
    query_vector=test_embedding_flat,  # Use 'query_vector' parameter directly
    limit=5  # Number of nearest neighbors you want to retrieve
)

In [7]:
# Step 5: Print the distances and indices of the nearest neighbors
for result in search_result_cosine:
    print(f"Distance: {result.score}")  # Score represents similarity (lower is better)
    print(f"Matched Title: {result.id}")
    print(f"Matched Url: {result.payload['url']}")
    print(f"Matched Section: {result.payload['section']}")

Distance: 0.9331667
Matched Title: 94
Matched Url: https://uscode.house.gov/view.xhtml?req=granuleid:USC-prelim-title18-section2259&num=0&edition=prelim
Matched Section: 2259. Mandatory restitution
Distance: 0.92769134
Matched Title: 201
Matched Url: https://uscode.house.gov/view.xhtml?req=granuleid:USC-prelim-title18-section2429&num=0&edition=prelim
Matched Section: 2429. Mandatory restitution
Distance: 0.9276248
Matched Title: 622
Matched Url: https://uscode.house.gov/view.xhtml?req=granuleid:USC-prelim-title18-section3594&num=0&edition=prelim
Matched Section: 3594. Imposition of a sentence of death
Distance: 0.92570907
Matched Title: 666
Matched Url: https://uscode.house.gov/view.xhtml?req=granuleid:USC-prelim-title18-section3663&num=0&edition=prelim
Matched Section: 3663. Order of restitution
Distance: 0.92277586
Matched Title: 599
Matched Url: https://uscode.house.gov/view.xhtml?req=granuleid:USC-prelim-title18-section3563&num=0&edition=prelim
Matched Section: 3563. Conditions of 

In [8]:
for result in search_result_euclid:
    print(f"Distance: {result.score}")  # Score represents similarity (lower is better)
    print(f"Matched Title: {result.id}")
    print(f"Matched Url: {result.payload['url']}")
    print(f"Matched Section: {result.payload['section']}")

Distance: 3.553698
Matched Title: 94
Matched Url: https://uscode.house.gov/view.xhtml?req=granuleid:USC-prelim-title18-section2259&num=0&edition=prelim
Matched Section: 2259. Mandatory restitution
Distance: 3.690732
Matched Title: 201
Matched Url: https://uscode.house.gov/view.xhtml?req=granuleid:USC-prelim-title18-section2429&num=0&edition=prelim
Matched Section: 2429. Mandatory restitution
Distance: 3.6949122
Matched Title: 622
Matched Url: https://uscode.house.gov/view.xhtml?req=granuleid:USC-prelim-title18-section3594&num=0&edition=prelim
Matched Section: 3594. Imposition of a sentence of death
Distance: 3.7389534
Matched Title: 666
Matched Url: https://uscode.house.gov/view.xhtml?req=granuleid:USC-prelim-title18-section3663&num=0&edition=prelim
Matched Section: 3663. Order of restitution
Distance: 3.8142586
Matched Title: 599
Matched Url: https://uscode.house.gov/view.xhtml?req=granuleid:USC-prelim-title18-section3563&num=0&edition=prelim
Matched Section: 3563. Conditions of proba